In [7]:
import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

In [8]:
train_input, train_target, test_input, test_target = \
prologue.load_data(one_hot_labels = True, normalize = True, flatten = False)
                                                                        
train_target.size()                                                                        
                                                                        

                                                                        
                                                                        

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:43: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/anaconda3/lib/python3.6/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


torch.Size([1000, 10])

In [9]:
"""class CNet(nn.Module):
    def __init__(self, num_classes):
        super(CNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=4)
        self.conv2 = nn.Conv2d(20, 800, kernel_size=5)
        self.fc1 = nn.Linear( 800*2*2,150)
        self.fc2 = nn.Linear(150,num_classes)
        
    def forward(self,x):
        x = F.torch.tanh(F.max_pool2d(self.conv1(x), kernel_size=2, stride=1))
        x = F.torch.tanh(F.max_pool2d(self.conv2(x), kernel_size=3, stride=1))
        x = F.torch.tanh(self.fc1(x.view(-1, 800*2*2)))
        x = F.softmax(self.fc2(x))
        return x"""

'class CNet(nn.Module):\n    def __init__(self, num_classes):\n        super(CNet, self).__init__()\n        self.conv1 = nn.Conv2d(1, 20, kernel_size=4)\n        self.conv2 = nn.Conv2d(20, 800, kernel_size=5)\n        self.fc1 = nn.Linear( 800*2*2,150)\n        self.fc2 = nn.Linear(150,num_classes)\n        \n    def forward(self,x):\n        x = F.torch.tanh(F.max_pool2d(self.conv1(x), kernel_size=2, stride=1))\n        x = F.torch.tanh(F.max_pool2d(self.conv2(x), kernel_size=3, stride=1))\n        x = F.torch.tanh(self.fc1(x.view(-1, 800*2*2)))\n        x = F.softmax(self.fc2(x))\n        return x'

In [10]:
class CNet(nn.Module):
    
    def __init__(self):
        
        super(CNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 20, kernel_size=4)
        self.conv2 = nn.Conv2d(20, 800, kernel_size=5)
        self.fc1 = nn.Linear(2*2*800,150)
        self.fc2 = nn.Linear(150, 10)

        
    def forward(self, x):
        
        # or use flatten
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3, stride = 3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size = 2, stride = 2))
        x = self.fc1(x.view(-1, 2*2*800))
        x = F.softmax(x)
        x = self.fc2(x)
        

        return x

In [11]:
def train_model(model, train_input, train_target, mini_batch_size):
    criterion = nn.MSELoss()
    eta = 1e-1
    epochs = 50
    
    for e in range(epochs):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            sum_loss = sum_loss + loss.item()
            for p in model.parameters():
                p.data.sub_(eta*p.grad.data)
        print(e, sum_loss)
        
        
def compute_nb_errors(model, input, target, mini_batch_size):
    for b in range (0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        predicted_classes = output.data.max(1)
        for k in range(mini_batch_size):
            if target.data[b + k, predicted_classes[k]] <= 0:
                nb_errors = nb_errors+1
    return nb_errors
                    
    
    

In [12]:
train_input, train_target = Variable(train_input), Variable(train_target)
test_input, test_target = Variable(test_input), Variable(test_target)

mini_batch_size = 50

In [96]:
#num_Classes  = 10
for k in range(10):
    model = CNet()
    train_model(model, train_input, train_target, mini_batch_size)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0 1.940513588488102
1 1.8605083972215652
2 1.825467862188816
3 1.8100897073745728
4 1.803306333720684
5 1.8002772852778435
6 1.798886500298977
7 1.798209398984909
8 1.7978424429893494
9 1.7976096719503403
10 1.7974344566464424
11 1.797283574938774
12 1.7971425876021385
13 1.797005295753479
14 1.7968689799308777
15 1.796732485294342
16 1.7965953201055527
17 1.796457163989544
18 1.7963179126381874
19 1.7961773574352264
20 1.7960354685783386
21 1.7958920449018478
22 1.7957469299435616
23 1.7956000044941902
24 1.7954511046409607
25 1.7952999845147133
26 1.7951464504003525
27 1.7949902266263962
28 1.7948310300707817
29 1.7946685403585434
30 1.794502355158329
31 1.7943319901823997
32 1.7941569313406944
33 1.7939765229821205
34 1.7937899604439735
35 1.7935962826013565
36 1.7933943271636963
37 1.7931824773550034
38 1.7929587587714195
39 1.7927205264568329
40 1.792464166879654
41 1.7921846881508827
42 1.7918748334050179
43 1.7915238812565804
44 1.79111497849226
45 1.7906204611063004
46 1.789992

RuntimeError: tensors used as indices must be long or byte tensors